In [1]:
import gspread
import pygsheets
from oauth2client.service_account import ServiceAccountCredentials
import plotly.express as px
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('esp32-smell-data-collection-4710923ae823.json', scope)
client = gspread.authorize(creds)

In [2]:
GG_sheet = client.open('TD5_ESP32_SMELL')
datasheet = GG_sheet.get_worksheet(0)

In [3]:
TEMP = []
Temp= datasheet.col_values(4)[2:]
Humid= datasheet.col_values(5)[2:]
MQ8_values= datasheet.col_values(6)[2:]
MQ9_values= datasheet.col_values(7)[2:]
MQ135_values= datasheet.col_values(8)[2:]
Date = datasheet.col_values(1)[2:]
Time = datasheet.col_values(2)[2:]
State = datasheet.col_values(3)[2:]
for i in range(len(Time)):
    TEMP.append(Date[i]+ " " +Time[i])

In [4]:
def pre_procesing(df):
    df = df[df['Temp'] >= 20]
    df = df[df['Humid'] >= 40]
    df = df[df['MQ8'] < 4095]
    df = df[df['MQ8'] > 0]
    df = df[df['MQ9'] < 4095]
    df = df[df['MQ9'] > 0]
    df = df[df['MQ135'] < 4095]
    df = df[df['MQ135'] > 0]    
    return df

In [5]:
Temp_datalist=[len(MQ8_values),len(MQ9_values),len(MQ135_values), len(Date), len(Time), len(Temp), len(Humid), len(TEMP)]
datas=[]
for i in range(min(Temp_datalist)):
    datas.append([Date[i], Time[i], State[i], pd.to_datetime(TEMP[i], dayfirst=True), float(Temp[i]), float(Humid[i]), int(MQ8_values[i]), int(MQ9_values[i]), int(MQ135_values[i])])
columns = ['Data', 'Time', 'State', 'TimeStep', 'Temp', 'Humid', 'MQ8', 'MQ9', 'MQ135'] 
data_sheet = pd.DataFrame(datas, columns=columns)
del datas

In [6]:
data_sheet=pre_procesing(data_sheet)
data_sheet=data_sheet.set_index('TimeStep')

In [7]:
data_sheet

,Data,Time,State,Temp,Humid,MQ8,MQ9,MQ135
TimeStep,,,,,,,,
2024-09-20 17:53:10,20/09/2024,17:53:10,Success,30.8,78.0,170,165,128
2024-09-20 17:53:19,20/09/2024,17:53:19,Success,30.6,78.0,165,163,126
2024-09-20 17:53:28,20/09/2024,17:53:28,Success,30.8,78.0,166,162,128
2024-09-20 17:53:37,20/09/2024,17:53:37,Success,30.5,78.0,162,160,127
2024-09-20 17:53:47,20/09/2024,17:53:47,Success,30.6,78.0,163,161,128
...,...,...,...,...,...,...,...,...
2024-10-08 16:44:18,08/10/2024,16:44:18,Success,31.5,40.0,142,159,132
2024-10-08 16:44:29,08/10/2024,16:44:29,Success,31.7,40.0,143,158,131
2024-10-08 16:44:41,08/10/2024,16:44:41,Success,31.6,40.0,142,157,130


In [8]:
data_sheet=pre_procesing(data_sheet)
data_sheet['MQ8_MA30'] = data_sheet['MQ8'].rolling(30).mean()
data_sheet['MQ9_MA30'] = data_sheet['MQ9'].rolling(30).mean()
data_sheet['MQ135_MA30'] = data_sheet['MQ135'].rolling(30).mean()
data_sheet['Temp_MA30'] = data_sheet['Temp'].rolling(30).mean()
data_sheet['Humid_MA30'] = data_sheet['Humid'].rolling(30).mean()
data_sheet.dropna(inplace=True)

In [9]:
data_sheet['DateTime'] = pd.to_datetime(data_sheet['Data'] + ' ' + data_sheet['Time'])
data_sheet['Data'] = pd.to_datetime(data_sheet['Data'])

In [10]:
import plotly.graph_objects as go 
import pandas as pd
#Loc du lieu ngay cua ngay can bieu dien
ngay_can_lay = '2024-09-21'

day = data_sheet[data_sheet["Data"].dt.date == pd.to_datetime(ngay_can_lay).date()]
time_col = day.index.time
mq_sensor = day[['MQ8', 'MQ9', 'MQ135','Temp','Humid','MQ8_MA30', 'MQ9_MA30', 'MQ135_MA30', 'Temp_MA30', 'Humid_MA30']]

In [11]:
mq_sensor

,MQ8,MQ9,MQ135,Temp,Humid,MQ8_MA30,MQ9_MA30,MQ135_MA30,Temp_MA30,Humid_MA30
TimeStep,,,,,,,,,,
2024-09-21 00:00:01,146,146,126,31.9,70.0,146.000000,146.000000,125.800000,31.823333,69.333333
2024-09-21 00:00:10,146,146,126,31.9,69.0,145.966667,146.000000,125.800000,31.820000,69.300000
2024-09-21 00:00:19,146,146,126,31.8,69.0,145.966667,145.966667,125.800000,31.823333,69.300000
2024-09-21 00:00:27,145,146,126,32.0,69.0,145.933333,145.933333,125.766667,31.833333,69.266667
2024-09-21 00:00:37,146,146,126,31.7,70.0,145.933333,145.933333,125.766667,31.833333,69.300000
...,...,...,...,...,...,...,...,...,...,...
2024-09-21 23:59:12,152,152,126,32.4,69.0,152.166667,151.800000,126.100000,32.306667,68.766667
2024-09-21 23:59:22,150,149,124,32.3,68.0,152.133333,151.733333,126.033333,32.310000,68.766667
2024-09-21 23:59:32,152,152,127,32.5,68.0,152.133333,151.766667,126.066667,32.316667,68.766667


## Trực quan hóa dữ liệu

In [12]:
fig_1 = go.Figure()
fig_1.add_scatter(x=mq_sensor.index,y=mq_sensor["MQ8_MA30"],name="MQ8",mode='lines')
fig_1.add_scatter(x=mq_sensor.index,y=mq_sensor["MQ9_MA30"],name="MQ9",mode='lines')
fig_1.add_scatter(x=mq_sensor.index,y=mq_sensor["MQ135_MA30"],name="MQ135",mode='lines')


fig_1.update_layout(
    title=f"Dữ liệu MQ của ngày {ngay_can_lay}",
    xaxis_title="Thời gian",
    yaxis_title="Giá trị MQ",
    legend_title="Cảm biến MQ",
)
fig_1.show()

In [13]:
from sklearn.preprocessing import MinMaxScaler # type: ignore
scaler = MinMaxScaler()
mq_sensor.columns
mq_scaler = mq_sensor[['MQ8', 'MQ9', 'MQ135', 'Temp', 'Humid']]
mq_scaler_fit = scaler.fit_transform(mq_scaler)
mq_pd = pd.DataFrame(mq_scaler_fit,columns=['MQ8', 'MQ9', 'MQ135', 'Temp', 'Humid'])

In [14]:
fig_hm = go.Figure(data=go.Heatmap(
    z = mq_pd.values,
    x= mq_pd.columns,
    y=time_col,
    colorscale='Plasma',
    xgap=10
))
fig_hm.update_layout(
    title=f'Biểu đồ nhiệt của MQ Sensor ngày {ngay_can_lay}',
    xaxis_title='MQ Sensors',
    yaxis_title='Time',
    yaxis_nticks=24
)

fig_hm.show()

In [15]:
#MQ8
b1 = 92.8636725
mq_sensor['MQ8_ME'] = ((mq_sensor['MQ8_MA30']-b1)/b1)*100
#MQ9
b2 = 93.6442624
mq_sensor['MQ9_ME'] = ((mq_sensor['MQ9_MA30']-b2)/b2)*100
#MQ135
b3 = 89.7450848
mq_sensor['MQ135_ME'] = ((mq_sensor['MQ135_MA30']-b3)/b3)*100
#Muc 0
tb1= 0
#Ve bieu do
fig_ss = go.Figure()
fig_ss.add_scatter(x=mq_sensor.index,y=mq_sensor['MQ8_ME'],name='MQ_8_ME',mode='lines')
fig_ss.add_scatter(x=mq_sensor.index, y=mq_sensor['MQ9_ME'], name='MQ9_ME', mode='lines')
fig_ss.add_scatter(x=mq_sensor.index, y=mq_sensor['MQ135_ME'], name='MQ135_ME', mode='lines')
fig_ss.add_scatter(x=mq_sensor.index, y=[tb1] * len(mq_sensor.index), name='Muc 0', mode='lines')

fig_ss.update_layout(
    title=f"Dữ liệu MQ của ngày {ngay_can_lay} so với dữ liệu đo được trong môi trường bình thường",
    xaxis_title="Thời gian",
    yaxis_title="Giá trị so sánh",
    legend_title="Cảm biến MQ"
)
fig_ss.show()